In [ ]:
# import Pkg
# Pkg.activate(".")

# pkgs = [
#     "Revise"
# ]
# Pkg.add(pkgs)
# for pkg in pkgs
#     eval(Meta.parse("import $pkg"))
# end

# ENV["LD_LIBRARY_PATH"] = ""

# Pkg.develop(path="../../..")
# # Pkg.develop(url="https://github.com/cjprybol/Mycelia.git")
# import Mycelia

In [ ]:
# data_directory = joinpath(dirname(pwd()), "data")

In [ ]:
# sample_directories = readdir(data_directory, join=true)

In [ ]:
reference_data_dir = mkpath(joinpath(dirname(pwd()), "reference_data"))

In [ ]:
# manually download and unzip

# target_genome = joinpath(reference_data_dir, "GCF_000001405.40_GRCh38.p14_genomic.fna")
# if !isdir(target_folder)
#     run(`curl -OJX GET "https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_000001405.40/download?include_annotation_type=GENOME_FASTA&filename=GCF_000001405.40.zip" -H "Accept: application/zip"`)
#     target_zip = joinpath(reference_data_dir, "GCF_000001405.40.zip")
#     mv("GCF_000001405.40.zip", target_zip)
#     run(`unzip $(target_zip)`)
# end
# mv(GCF_000001405.40_GRCh38.p14_genomic.fna)

In [ ]:
# conda create -n bwa-mem2 -c bioconda bwa-mem2
srun --partition=interactive --account=mpsnyder --time=24:00:00 --nodes=1 --ntasks=1 --cpus-per-task=8 conda run --no-capture-output -n bwa-mem2 bwa-mem2 index $HOME/workspace/Mycelia/projects/ME-CFS/reference_data/ncbi_dataset/data/GCF_000001405.40/GCF_000001405.40_GRCh38.p14_genomic.fna
bwa-mem2-2.2.1_x64-linux/bwa-mem2 mem ref.fa read1.fq read2.fq > out.sam

In [ ]:
for sample_directory in sample_directories[2:end]
    fastq_files = filter(x -> occursin(r"\.fq\.gz$", x), readdir(sample_directory, join=true))
    sample_ids = unique(replace.(basename.(fastq_files), r"_[12]\.fq\.gz" => ""))
    for sample_id in sample_ids
        forward = joinpath(sample_directory, sample_id * "_1.fq.gz")
        reverse = joinpath(sample_directory, sample_id * "_2.fq.gz")
        output_directory = joinpath(sample_directory, sample_id * "_trimgalore")
        run(`sbatch trim-galore.sh $(output_directory) $(forward) $(reverse)`)
    end
end